In [ ]:
import numpy as np
import pandas as pd
import math
import sys
import random
from ipynb.fs.full.Formulas import *
from ipynb.fs.full.Environment import *
from ipynb.fs.full.Flow_Grouping_Algo import *

In [ ]:
def flows_classifier():
    set_of_direct_flows = set()
    set_of_relay_flows = set()
    candidate_relay_set_for_relay_flows = {}
    for i in range(1,number_of_flows+1):
        relay_type, candidate_relay_set, _ = flows_candidate_relay_set_pairs[str([i,0.0])]
        if relay_type == 'dont_relay':
            set_of_direct_flows.add(i)
        else:
            set_of_relay_flows.add(i)
            candidate_relay_set_for_relay_flows[i] = candidate_relay_set
    return set_of_direct_flows, set_of_relay_flows, candidate_relay_set_for_relay_flows

In [ ]:
set_of_direct_flows, set_of_relay_flows, candidate_relay_set_for_relay_flows = flows_classifier()

In [ ]:
def DGroup(Va, Ea, G):
    # Obtain edges among Va and existing relay flows in group G
    Edr = get_edges_among_Va_and_G(Va, G)
    E = Ea + Edr
    DG = generateGroup(Va, E)
    return DG

def RGroup(Vr, Rg, G):
    RG = set()
    for v in Vr:
        for r in Rg[v]:
            if not is_collided(v, r, G + RG):
                RG.add(r)
                break
    return RG

def JRDS(Va, Ea, Vr, Rg, M):
    G = set()
    m = 0
    U = 0

    while True:
        m += 1
        # Update the coordinates of vehicles and UAVs
        update_coordinates(Va, Vr)
        
        # Obtain optional relay set for each flow
        Rg = obtain_optional_relay_set(Va, Vr)

        # Check if some flow in G is newly completed
        if some_flow_completed_in_G(G):
            # Obtain completed flows in G'
            G_prime = obtain_completed_flows(G)
            # Update G
            G -= G_prime
            Va -= G_prime  # Update Va
            Vr -= G_prime  # Update Vr

        # Form new groups for flows that can transmit concurrently
        G += DGroup(Va, Ea, G)
        G += RGroup(Vr, Rg, G)

        # Check if all flows have been accomplished
        if all_flows_completed(Va, Vr):
            break

    U = calculate_system_throughput(G, M)
    return G, m, U
